In [1]:
import pprint
import pandas as pd
import numpy as np
import re
import ast
import warnings
warnings.filterwarnings('ignore')

# Price

In [2]:
# rates = {'€': 1, '£': 0.86, 'zł': 4.48, 'SAR': 4.01, "'UZS': 12256.20", "'VND': 25120.40", '¥': 150.03, 'US$': 1.07, 'MAD': 10.93,
#          "'MYR': 4.91", '4.89': 1, 'PKR': 307.51, '₪': 3.9928904, 'CHF': 0.97, 'BZD': 2.17, 'HK$': 8.3801351, '₱': 60.0592, 
#          'DZD': 146.16, "'Rp': 15901.44", 'GTQ': 8.43, 'AED': 3.93, 'CAD': 1.44, 'NPR': 141.71, 'R$': 5.32, 'PEN': 3.97, 
#          'MXN': 18.79, "'PYG': 7734.95", 'AUD': 1.62, 'THB': 37.31, 'BDT': 115.48, 'XOF': 655.83, 'EGP': 33.03, 'TL': 22.68, 
#          'ZAR': 20.73, 'NOK': 11.85, 'MKD': 61.49, 'SEK': 11.64, 'UAH': 39.76, 'AMD': 415.06, 'DKK': 7.45, 'BOB': 7.44, 
#          'SCR': 14.13, "'COP': 4516.99", 'RUB': 86.86, '$': 1.07, '₹': 88.4, "'HUF': 368.77", 'DOP': 58.35, 'NGN': 494.81, 
#          "'LAK': 19367.90", 'KHR': 4415.98, 'CRC': 576, 'MDL': 19.09, 'KES': 148.79, 'TND': 3.31, "'TZS': 2541.61", 'MZN': 67.75, 
#          'lei': 4.97, 'AZN': 1.82, 'HNL': 26.48, 'KWD': 0.33, 'UYU': 41.58, 'XAF': 657.29, 'JOD': 0.76, 'ALL': 108.68, 
#          'BGN': 1.96, 'S$': 0.6917, 'BYN': 2.7, 'PAB': 1.07, 'NIO': 39.16, "'KRW': 1393.32", 'Kč': 23.52, 'LYD': 5.18, 
#          'RSD': 117.22, "'MNT': 3726.17", 'QAR': 3.9, 'TWD': 32.9}

In [3]:
# rates = {"€":1,"£":0.86,"zł":4.48,"SAR":4.01,"UZS":12.300,"VND":25.123,"¥":150.03,"US$":1.07,"MAD":10.93,"MYR":1,"4.89":1,
#          "PKR":307.51,"₪":3.9928904,"CHF":0.97,"BZD":2.17,"HK$":8.3801351,"₱":60.0592,"DZD":146.16,"Rp":15.899,"GTQ":8.43,
#          "AED":3.93,"CAD":1.44,"NPR":141.71,"R$":5.32,"PEN":3.97,"MXN":18.79,"PYG":7.793,"AUD":1.62,"THB":37.31,"BDT":115.48,
#          "XOF":655.83,"EGP":33.03,"TL":22.68,"ZAR":20.73,"NOK":11.85,"MKD":61.49,"SEK":11.64,"UAH":39.76,"AMD":415.06,"DKK":7.45,
#          "BOB":7.44,"SCR":14.13,"COP":4.652,"RUB":86.86,"$":1.07,"₹":88.40,"HUF":36910,"DOP":58.35,"NGN":494.81,"LAK":19.251}

In [4]:
def price_cleaning_converting(df):
    
    # We used a proxi rotation API to scrape the data from the site,proxi uses different IP's in each request,
    # each scrape had different price currency,made a dictionary with the unique values of df['signs'] column
    #and their convert rate to euro (5 June 2023 convert rates).
    rates = {'€': 1, '£': 0.86, 'zł': 4.48, 'SAR': 4.01, 'UZS': 12256.20, 'VND': 25120.40, '¥': 150.03, 'US$': 1.07, 'MAD': 10.93,
         'MYR': 4.91, '4.89': 1, 'PKR': 307.51, '₪': 3.9928904, 'CHF': 0.97, 'BZD': 2.17, 'HK$': 8.3801351, '₱': 60.0592, 
         'DZD': 146.16, 'Rp': 15901.44, 'GTQ': 8.43, 'AED': 3.93, 'CAD': 1.44, 'NPR': 141.71, 'R$': 5.32, 'PEN': 3.97, 
         'MXN': 18.79, 'PYG': 7734.95, 'AUD': 1.62, 'THB': 37.31, 'BDT': 115.48, 'XOF': 655.83, 'EGP': 33.03, 'TL': 22.68, 
         'ZAR': 20.73, 'NOK': 11.85, 'MKD': 61.49, 'SEK': 11.64, 'UAH': 39.76, 'AMD': 415.06, 'DKK': 7.45, 'BOB': 7.44, 
         'SCR': 14.13, 'COP': 4516.99, 'RUB': 86.86, '$': 1.07, '₹': 88.4, 'HUF': 368.77, 'DOP': 58.35, 'NGN': 494.81, 
         'LAK': 19367.90, 'KHR': 4415.98, 'CRC': 576, 'MDL': 19.09, 'KES': 148.79, 'TND': 3.31, 'TZS': 2541.61, 'MZN': 67.75, 
         'lei': 4.97, 'AZN': 1.82, 'HNL': 26.48, 'KWD': 0.33, 'UYU': 41.58, 'XAF': 657.29, 'JOD': 0.76, 'ALL': 108.68, 
         'BGN': 1.96, 'S$': 0.6917, 'BYN': 2.7, 'PAB': 1.07, 'NIO': 39.16, 'KRW': 1393.32, 'Kč': 23.52, 'LYD': 5.18, 
         'RSD': 117.22, 'MNT': 3726.17, 'QAR': 3.9, 'TWD': 32.9, "BAM": 1.96, "TTD": 7.26, "LKR": 313.28, "MUR": 48.81, "GEL": 2.80,
         "OMR": 0.41, "JMD": 166.05, "IRR": 45330.16, "BHD": 0.40, "XCD": 2.90}

    #Cleaning
    df['price'] = df['price'].replace(r"<span class=\"bui-u-sr-only\">\nOriginal price\n","",regex=True)
    df['price'] = df['price'].replace(r"<span class=\"bui-u-sr-only\">\n","",regex=True)
    df["price"]=df["price"].replace(r"\n</span>","",regex=True)
    df["price"]=df["price"].replace(".*Current price","",regex=True)
    df["price"]=df["price"].replace(".*\n","",regex=True)
    df["signs"] = df["price"].replace("[\d]","",regex=True)
    df["signs"] = df["signs"].replace("[\,\"]","",regex=True)
    df["price_clean"]=df["price"].replace("[^\d]","",regex=True)
    
    
    # PREPROCCESS TO APPLY THE DICTIONARY AND CONVERT THE PRICES TO THE NEW COLUMN #
    
    #Fill the Nan in Signs column with the most frequent value
    df["signs"] = df["signs"].fillna(df['signs'].value_counts().idxmax())
    #Fill Nans in price
    df["price_clean"] = df["price_clean"].fillna('0')
    #Convert price to integer
    df["price_clean"] = df["price_clean"].astype("int")
    #Replace unicode spaces in signs column with nothing
    df['signs'] = df['signs'].str.replace('\xa0', '')
    #Strip
    df["signs"] = df['signs'].str.strip()
    
    #APPLY
    #O KWDIKAS MEXRI TORA LITOURGEI EAN STO DICTIONARY_RATES YPARXOUN TA SYMBOLA TOY COLUMN SIGNS, ALLIOS PETAEI ERROR.MIPWS
    #PREPEI NA KANOUME TRY?
    df['amount_conv'] = df.apply(lambda x: x.price_clean / rates[x.signs], axis=1 )
    
    return df



# Reviews and Rating

In [5]:
#"Reviews" column Cleaning and creating two new columns for the corresponding values.

def reviews_df(df):
    df["TestRev"] = df["reviews"].replace("[A-Za-z]","", regex = True)
    df["TestRev"].str.split(" ",1, expand = True)
    df[["Rating","NumReviews"]] = df["TestRev"].str.split(" ",1, expand = True)
    #df.drop(columns=["reviews","TestRev"],inplace = True)
    
    return df



# Facilities

In [6]:
def facilities(df):
    
    #Create new list with all splitted values in each cell in facilities
    
    list_facilities = []
    for lst in df['facilities']:
        try:
            for elem in lst.split(","):
                list_facilities.append(elem)
            
        except:
            pass
    
    #make the list set
    set_facilities = set(list_facilities)
    
    #Clean the special characters from the set
    facilities_list = [re.sub(r"[^\w\s]", "", item) for item in set_facilities]
    
    #Convert to List Function
    def convert_to_list(value):
        try:
            return ast.literal_eval(value)
        except (SyntaxError, ValueError):
            return np.nan
    
    #Convert the elements inside each cell in list.
    df['facilities2'] = df['facilities'].apply(convert_to_list)
    
    df.dropna(subset = "facilities2",inplace=True)
    
    #Strip the elements inside each cell
    df["facilities2"] =df["facilities2"].apply(lambda x: [element.strip()for element in x])
    

    #Pools, Airport shuttle, Breakfast, Family Rooms, Parking, Restaurant. New columns for these with boolean.
    
    df["Airport"] = df["facilities2"].apply(lambda x: any(re.search(r"\b[Aa]irport\b", elem) for elem in x))
    df["Pool"] = df["facilities2"].apply(lambda x: any(re.search(r"\b[Pp]ool\b", elem) for elem in x))
    df["Breakfast"] = df["facilities2"].apply(lambda x: any(re.search(r"\b[Bb]reakfast\b", elem) for elem in x))
    df["Family_Rooms"] = df["facilities2"].apply(lambda x: any(re.search(r"\b[Ff]amily\b", elem) for elem in x))
    df["Parking"] = df["facilities2"].apply(lambda x: any(re.search(r"\b[Pp]arking\b", elem) for elem in x))
    df["Restaurant"] = df["facilities2"].apply(lambda x: any(re.search(r"\b[Rr]estaurant\b", elem) for elem in x))
    
    
    return df  

# Size

In [7]:
def clean_df_size(df):
    df[["size_num","size_meaus_unit"]] = df['size'].str.split(" ",2,expand=True)
    df['Feet_num'] = df.size_num.loc[df.size_meaus_unit.eq('feet²')]
    df['Feet_num'] = pd.to_numeric(df["Feet_num"])
    df["Feet_num"] = df["Feet_num"]//10.76391041671
    df['M_num'] = df.size_num.loc[df.size_meaus_unit.eq('m²')]
    df['M_num'] = pd.to_numeric(df["M_num"])
    # np.select and define list of condition with corresponding values
    df['size_m2']=(np.select([df['size_meaus_unit'].eq('m²'), 
                         df['size_meaus_unit'].eq('feet²')],
                        [df['M_num'],
                         df['Feet_num']],
                        np.nan)) 
    df.drop(["size","size_num","size_meaus_unit","Feet_num","M_num"],axis=1 ,inplace=True)
    return df

# Pets

In [8]:
def Pets(df):
    df["Pets"] = df["rules"].apply(lambda x: "Pets are allowed" in x)
    return  df

### Start to clean the csvs

In [9]:
# df = pd.read_csv("crete_June_3_stars_0_50_150_200.csv")
# df1 = pd.read_csv("crete_June_3_stars_without_apartments.csv")
# df2 = pd.read_csv("crete_June_3stars_0_50_100.csv")
# df3 = pd.read_csv("Crete_June_4starApartment2.csv")
# df4 = pd.read_csv("CreteJune4_WithoutApartments.csv")
# df5 = pd.read_csv("CreteJune125.csv")
# df6 = pd.read_csv("CreteJuneUnrated.csv")
#lstdf = [df,df1,df2,df3,df4,df5,df6]

In [10]:
LefkadaJune = pd.read_csv("LefkadaJune.csv")
MykonosJune = pd.read_csv("MykonosJune.csv")
NaxosJune = pd.read_csv("NaxosJune.csv")
ParosJune = pd.read_csv("ParosJune.csv")
RhodesJune = pd.read_csv("RhodesJune.csv")
Samos_June = pd.read_csv("Samos_June.csv")
SantoriniJune = pd.read_csv("SantoriniJune.csv")
Tinos_June = pd.read_csv("Tinos_June.csv")

In [11]:
#RhodesJune2

In [12]:
dicidf = {"Lefkada":LefkadaJune,
         "Mykonos":MykonosJune,
         "Naxos":NaxosJune,
         "Paros":ParosJune,
         "Rhodes":RhodesJune,
         "Samos":Samos_June,
         "Santorini":SantoriniJune,
         "Tinos":Tinos_June}

In [13]:
for name,d in dicidf.items():
    d.dropna(subset=["rules"],inplace = True)
    d = price_cleaning_converting(d)
    d = reviews_df(d)
    d = facilities(d)
    d = clean_df_size(d)
    d = Pets(d)
    try:
        d.drop(columns=["link"],inplace=True)
    except:
        pass
    d.drop(columns = ["facilities","price","rules","signs","price_clean","facilities2","reviews","TestRev"],inplace=True)
    d["Month"] = "June"
    d["Location"] = f"{name}"
    d.to_csv(f"{name}.csv",index=False)